In [17]:
import pandas as pd
import nltk
import spacy
from tqdm import tqdm

## CAP

In [3]:
CAP=pd.read_json("../data/raw/CAP/concat/casebody.json")

In [9]:
CAP["0"][0]["data"].keys()

dict_keys(['judges', 'head_matter', 'corrections', 'opinions', 'attorneys'])

In [13]:
CAP["0"][0]["data"]["opinions"][0]["text"]

'Per Curiam.\nAppellant, James Joseph Standley, Jr., by his attorney, has filed for a rule on the clerk. His attorney, Billy J. Allred, admits that the record was tendered late due to a mistake on his part.\nWe find that such error, admittedly made by the attorney for a criminal defendant, is good cause to grant the motion. See our Per Curiam opinion dated February 5, 1979, In Re: Belated Appeals in Criminal Cases.\nA copy of this opinion will be forwarded to the Committee on Professional Conduct.'

0    Per Curiam.\nAppellant, James Joseph Standley,...
1    Per Curiam.\nPetitioner Keith Melvin Dubray, b...
2    Per Curiam.\nAppellant, Avery Nathan Richardso...
3    Per Curiam.\nAppellant, Sammy Joe Elmore, by h...
4    George Rose Smith, Justice.\nThe two appellant...
Name: 0, dtype: object

In [29]:
def extract_opinion(x):
    try:
        opinion=x["data"]["opinions"][0]["text"]
    except Exception as e:
        opinion=""
    return opinion

In [30]:
CAP["opinions"]=CAP["0"].apply(lambda x:extract_opinion(x))

In [31]:
CAP["0"][0]["data"]["opinions"][0]["text"]

'Per Curiam.\nAppellant, James Joseph Standley, Jr., by his attorney, has filed for a rule on the clerk. His attorney, Billy J. Allred, admits that the record was tendered late due to a mistake on his part.\nWe find that such error, admittedly made by the attorney for a criminal defendant, is good cause to grant the motion. See our Per Curiam opinion dated February 5, 1979, In Re: Belated Appeals in Criminal Cases.\nA copy of this opinion will be forwarded to the Committee on Professional Conduct.'

0         Per Curiam.\nAppellant, James Joseph Standley,...
1         Per Curiam.\nPetitioner Keith Melvin Dubray, b...
2         Per Curiam.\nAppellant, Avery Nathan Richardso...
3         Per Curiam.\nAppellant, Sammy Joe Elmore, by h...
4         George Rose Smith, Justice.\nThe two appellant...
                                ...                        
358701    OPINION\nBOSSON, J.\n{1} Charlie Taylor (Defen...
358702    OPINION\nWECHSLER, Judge.\n{1} Plaintiffs-Appe...
358703    OPINION\nBOSSON, J.\n{1} We consider as a matt...
358704    OPINION\nPICKARD, Chief Judge.\n{1} This ease ...
358705    OPINION\nBUSTAMANTE, Judge.\n{1} Angela V. Woo...
Name: opinions, Length: 358706, dtype: object

In [36]:
CAP_op=CAP["opinions"].to_list()

In [39]:
print(CAP_op[0])

Per Curiam.
Appellant, James Joseph Standley, Jr., by his attorney, has filed for a rule on the clerk. His attorney, Billy J. Allred, admits that the record was tendered late due to a mistake on his part.
We find that such error, admittedly made by the attorney for a criminal defendant, is good cause to grant the motion. See our Per Curiam opinion dated February 5, 1979, In Re: Belated Appeals in Criminal Cases.
A copy of this opinion will be forwarded to the Committee on Professional Conduct.


In [40]:
with open("../data/processed/CAP/CAP.txt","w") as file:
    file.writelines(CAP_op)

## GPPD

In [45]:
GPPD=pd.read_csv("../data/raw/GPPD/patent.csv")

In [48]:
GPPD.to_csv("../data/processed/GPPD/GPPD.txt",header=None,index=None,sep="\n")

## PL

In [ ]:
GPPD=pd.read_csv("../data/raw/PL/long_descrip.txt")

## Ensemble

In [2]:
with open ("../data/processed/Ensemble/Ensemble_v0.txt","r") as file:
    Ensemble = file.read()

## Sentence Segmentation

In [7]:
l=int(len(Ensemble)/6)

In [11]:
l

817248356

In [12]:
# ran ge(40) needs more than 300G RAM

Ensemble_trunks=[Ensemble[l*n:l*(n+1)] for n in range(6)]

In [14]:
for i in range(len(Ensemble_trunks)):
    with open("../data/processed/Ensemble_splits/Ensemble_split{}.txt".format(i),"w") as file:
        file.write(Ensemble_trunks[i])

In [4]:
# Ensemble_trunk0=Ensemble[0:2**30-1]
# Ensemble_trunk1=Ensemble[2**30-1:2*2**30-2]
# Ensemble_trunk2=Ensemble[2*2**30-2:3*2**30-3]
# Ensemble_trunk3=Ensemble[3*2**30-3:4*2**30-4]
# Ensemble_trunk4=Ensemble[4*2**30-4:5*2**30-5]

In [14]:
texts="hello fash foi aw foaehw ifhweua of weaoufh uew hefou awh"

In [26]:
import spacy
from spacy.lang.en import English

nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)
nlp.max_length=2**30
# nlp.disable_pipes(["tagger", "parser", "ner"])
for i in range(len(Ensemble_trunks)):
    trunk=Ensemble_trunks[i]
    doc = nlp(trunk)
    with open("../data/processed/Ensemble/Ensemble_trunk{}.txt".format(i),"w") as file:
        for sent in doc.sents:
            file.write(sent.text+"\n")

[sajf jfkjsa]

In [ ]:
import multiprocessing
import spacy
from spacy.lang.en import English
import pickle

def worker(procnum, return_dict,trunk):
    """worker function"""
    return_dict[procnum] =[str(sent)+"\n" for sent in list(nlp(trunk))]

nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)
nlp.max_length=2**30

manager = multiprocessing.Manager()
return_dict = manager.dict()
jobs = []
for i in range(20):
    with open("../data/processed/Ensemble/Ensemble_trunk{}.txt".format(i),"r") as file:
        trunk=file.read()
    p = multiprocessing.Process(target=worker, args=(i, return_dict,trunk))
    jobs.append(p)
    p.start()

for proc in jobs:
    proc.join()

with open("sent_segmentation.pkl", "wb") as f:
    pickle.dump(return_dict, f)

## Shuffle & split

In [18]:
import numpy as np
from sklearn.model_selection import train_test_split

In [19]:
with open("../data/processed/Ensemble/Ensemble_sent_segmented.txt","r") as file:
    sentence = file.read().splitlines()

sent_df=pd.DataFrame(sentence,columns=["sent"])

In [32]:
# drop rows containing only \n or space
sent_df=sent_df[sent_df['sent'].apply(lambda x:len(x.strip())>0)]

In [33]:
X_train, X_test = train_test_split(sent_df, test_size=0.05, random_state=0)

X_val,X_test = train_test_split(X_test, test_size=0.5, random_state=0)


In [36]:
len(X_train),len(X_test),len(X_val)

(39571621, 1041359, 1041358)

In [38]:

X_train.to_csv("../data/processed/Ensemble/legal.{}.txt".format("train"), sep='\n', index=False,header=None)
# X_test.to_csv("../data/processed/Ensemble/legal.{}.txt".format("test"), sep='\n', index=False,header=None)
X_val.to_csv("../data/processed/Ensemble/legal.{}.txt".format("valid"), sep='\n', index=False,header=None)

In [ ]:
for split in ["train","test","valid"]:
    with open("../data/processed/Ensemble/legal.{}.txt".format(split),"w") as file:
            file.write(Ensemble_trunks[i])

In [24]:
import spacy
from spacy.lang.en import English

nlp = English()  # just the language with no model
sentencizer = nlp.create_pipe("sentencizer")
nlp.add_pipe(sentencizer)

trunk="This is a sentence. This is another sentence."
doc = nlp(trunk)

In [38]:
with open("./test.txt","w") as file:
    for sent in doc.sents:
        file.write(sent.text+"\n")

In [36]:
dir(spacy.tokens.span.Span)

['_',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_fix_dep_copy',
 '_recalculate_indices',
 '_vector',
 '_vector_norm',
 'as_doc',
 'char_span',
 'conjuncts',
 'doc',
 'end',
 'end_char',
 'ent_id',
 'ent_id_',
 'ents',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'kb_id',
 'kb_id_',
 'label',
 'label_',
 'lefts',
 'lemma_',
 'lower_',
 'merge',
 'n_lefts',
 'n_rights',
 'noun_chunks',
 'orth_',
 'remove_extension',
 'rights',
 'root',
 'sent',
 'sentiment',
 'set_extension',
 'similarity',
 'start',
 'start_char',
 'string',
 'subtree',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'upper_',
 'vector',
 'vector_norm'